<a href="https://colab.research.google.com/github/konstantinstanmeyer/music-extractor-1/blob/main/find_notes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# imports

import os # validates if audio files already exist locally


In [9]:
# file downloads

# audio file name, without file extension
current_song = 'wish-you-were-here'

# possibly remove boolean check once jupyter notebook's behavior is figured out

if not os.path.isfile("{current_song}.wav".format(current_song=current_song)):
  print("file downloads starting...")

  # match import to current_song
  !wget https://github.com/konstantinstanmeyer/music-extractor-files/raw/main/wish-you-were-here.m4a
  !ls

  # convert file type(s) to wav // check if this is necessary // add alternate file type handling once
  if os.path.isfile("{current_song}.m4a".format(current_song=current_song)):
    !ffmpeg -i {current_song}.m4a {current_song}.wav

file downloads starting...
--2023-10-31 21:15:29--  https://github.com/konstantinstanmeyer/music-extractor-files/raw/main/wish-you-were-here.m4a
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/konstantinstanmeyer/music-extractor-files/main/wish-you-were-here.m4a [following]
--2023-10-31 21:15:29--  https://raw.githubusercontent.com/konstantinstanmeyer/music-extractor-files/main/wish-you-were-here.m4a
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 484583 (473K) [audio/mp4]
Saving to: ‘wish-you-were-here.m4a’

wish-you-were-here. 100%[===================>] 473.23K  2.99MB/s    in 0.2s    

2023

In [6]:
!ls

sample_data


In [ ]:
# initializing key values: define fft windows,

# FUTURE: add note start times, then define windows to be 100ms(likely something a slight bit larger)
FFT_WINDOW_SECONDS = 0.25

In [2]:
!pip install pydub

In [17]:
# finding note starts
# code adjusted from Ian Vonseggern's NotePrediction Project: https://github.com/ianvonseggern1/note-prediction/blob/master/note_recognition.py
# big-boy thanks! https://www.linkedin.com/in/ian-vonseggern-83699b154/

from pydub import AudioSegment

def predict_note_starts(song):
    # Size of segments to break song into for volume calculations
    SEGMENT_MS = 50
    # Minimum volume necessary to be considered a note
    VOLUME_THRESHOLD = -35
    # The increase from one sample to the next required to be considered a note
    EDGE_THRESHOLD = 5
    # Throw out any additional notes found in this window
    MIN_MS_BETWEEN = 100

    # Filter out lower frequencies to reduce noise
    # song = song.high_pass_filter(80)
    # dBFS is decibels relative to the maximum possible loudness
    volume = [segment.dBFS for segment in song[::SEGMENT_MS]]

    predicted_starts = []
    for i in range(1, len(volume)):
        if volume[i] > VOLUME_THRESHOLD and volume[i] - volume[i - 1] > EDGE_THRESHOLD:
            ms = i * SEGMENT_MS
            # Ignore any too close together
            if len(predicted_starts) == 0 or ms - predicted_starts[-1] >= MIN_MS_BETWEEN:
                predicted_starts.append(ms)

    return predicted_starts, song

    # If actual note start times are provided print a comparison
    # if len(actual_starts) > 0:
    #     print("Approximate actual note start times ({})".format(len(actual_starts)))
    #     print(" ".join(["{:5.2f}".format(s) for s in actual_starts]))
    #     print("Predicted note start times ({})".format(len(predicted_starts)))
    #     print(" ".join(["{:5.2f}".format(ms / 1000) for ms in predicted_starts]))

In [18]:
song = AudioSegment.from_file("wish-you-were-here.wav")

predict_note_starts(song)

([350,
  850,
  1300,
  2250,
  4250,
  4750,
  5200,
  6200,
  7650,
  8100,
  8550,
  9050,
  10050,
  12000,
  12950,
  13900,
  15350,
  15800,
  16300,
  16800,
  17750,
  19200,
  19700,
  20550,
  21300,
  21550,
  23450,
  23950,
  24400,
  25350],
 <pydub.audio_segment.AudioSegment at 0x7ee96bfc1240>)